In [450]:
from __future__ import annotations
import random
from enum import Enum, IntEnum, auto
from dataclasses import dataclass
from collections import Counter
from more_itertools import always_iterable

class Suit(Enum):
    spade = "♠"
    diamond = "♦"
    heart = "♥"
    club = "♣"

class Rank(IntEnum):
    TWO = 2
    THREE = 3
    FOUR = 4
    FIVE = 5
    SIX = 6
    SEVEN = 7
    EIGHT = 8
    NINE = 9
    TEN = 10
    JACK = 11
    QUEEN = 12
    KING = 13
    ACE = 14

class Hands(IntEnum):
    HIGH = auto()
    PAIR = auto()
    TWOPAIR = auto()
    THREEOFAKIND = auto()
    STRAIGHT = auto()
    FLUSH = auto()
    FULLHOUSE = auto()
    FOUROFAKIND = auto()
    STRAIGHTFLUSH = auto()


@dataclass
class Card:
    rank: Rank
    suit: Suit

    def __repr__(self):
        return f"|{self.rank}{self.suit.value}|" if self.rank < Rank.JACK \
            else f"|{self.rank.name[0]}{self.suit.value}|"
    def __gt__(self, other) -> bool:
        return self.rank > other.rank
        
class Hand:
    def __init__(self, cards: list[Card]=None):
        self.cards = list(always_iterable(cards))
    def fold(self):
        self.cards = []
    def show(self):
        return self.cards
    def __repr__(self):
        return str(self.cards)

class Deck:
    def __init__(self):
        self.cards = [Card(rank=rank, suit=suit) for rank in Rank for suit in Suit]
        self.shuffle()
    def shuffle(self):
        random.shuffle(self.cards)
    def deal(self, n=1) -> Card:
        return [self.cards.pop() for _ in range(n)]

deck = Deck()

In [451]:
class Game:
    def __init__(self, deck:Deck, *, small_blind:int = 1, big_blind:int = 2, limit:bool = None):
        self.deck = deck
        self.community_cards = []
        self.players = []
    def flip(self):
        if not len(self.community_cards): # flop
            self.community_cards += self.deck.deal(3)
        elif len(self.community_cards) < 5: # turn, river
            self.community_cards += self.deck.deal(1)
        else:
            raise ValueError("Too many cards! This ain't Texas Hold'em no more!")
    @property
    def phase(self):
        return {0: "deal", 3: "flop", 4: "turn", 5: "river"}.get(len(self.community_cards))
    def winner(self):
        return max(self.players, key=lambda player: player.best_hand) #TODO this doesn't deal with ties / multiple winners
    def __repr__(self):
        players = "\n* ".join(map(str, sorted(self.players, reverse=True)))
        return f"""Pot: X | Bet: X | Phase: {self.phase.capitalize()}
Community Cards: {self.community_cards}
---------------------------------------
Players: 
* {players}
"""

In [452]:
def has_pairs(cards: list[Card]) -> tuple[Hands, list[Card]] | None:
    """Determine if the set of cards has pairlike hands and return the pair type
    
    This would be used in conjunction with a sorted hand to determine the winner of a game"""
    pairs = Counter(card.rank for card in cards)
    if (four:=pairs.most_common(1)[0])[1] == 4:
        rank = four[0]
        hand = sorted([card for card in cards if card.rank==rank]) \
            + sorted([card for card in cards if card.rank!=rank], reverse=True)[1]
        return Hands.FOUROFAKIND, hand
    else:
        pairs = {card: count for card, count in pairs.items() if count >=2}
        pairscore = sum(pairs.values())
        hand_type = {0:Hands.HIGH, 2:Hands.PAIR, 3:Hands.THREEOFAKIND, 4:Hands.TWOPAIR, 5:Hands.FULLHOUSE}.get(pairscore)
        hand = sorted([card for card in cards if card.rank in pairs]) \
            + sorted([card for card in cards if card.rank not in pairs], reverse=True)[:5-pairscore]
        return hand_type, hand
        
def has_flush(cards: list[Card]) -> tuple[Hands, list[Card]] | None:
    """Determine if the cards are a flush and return the hand
    
    In this case a flush hand is the 5 highest cards in the same suit"""
    suits = Counter(card.suit for card in cards)
    suit, count = suits.most_common(1)[0]
    if count >=5: # a flush!
        hand = sorted((card for card in cards if card.suit == suit), reverse=True)
        return (Hands.FLUSH, hand[:5])
    else:
        return None

def drop_none(l: list) -> list:
    return [item for item in l if item]

In [459]:
class Player:
    def __init__(self, name:str, *, game:Game, buy:int = 1000):
        self.chips = buy
        self.name = name
        self.game = game
        self.hand = Hand(self.game.deck.deal(2))
        self.game.players += [self]
    @property
    def cards(self):
        return self.hand.cards + self.game.community_cards
    def bid(self, amount:int):
        self.chips -= amount # this won't really work ...
    def draw(self, deck:Deck):
        self.hand.cards += deck.deal(1)
    def fold(self):
        self.hand.fold()
        self.game.players.remove(self)
        
    @property
    def best_hand(self) -> tuple(Hands, list[Card]):
        return max(drop_none([has_flush(self.cards), has_pairs(self.cards)]), key=lambda hand_rank: hand_rank[0])
    def __repr__(self):
        #return f"[{self.name} (${self.chips})] {self.cards} --> {self.best_hand}"
        return f"[{self.name} (${self.chips})] {self.best_hand}"
    def __gt__(self, other) -> bool:
        return self.best_hand > other.best_hand

In [460]:
game = Game(deck=Deck())
players = [Player(name.capitalize(), game=game) for name in ["liz","Robert","lucas","gio","taylor"]]
game

Pot: X | Bet: X | Phase: Deal
Community Cards: []
---------------------------------------
Players: 
* [Lucas ($1000)] (<Hands.HIGH: 1>, [|K♠|, |3♣|])
* [Liz ($1000)] (<Hands.HIGH: 1>, [|Q♦|, |8♦|])
* [Robert ($1000)] (<Hands.HIGH: 1>, [|Q♥|, |8♥|])
* [Gio ($1000)] (<Hands.HIGH: 1>, [|Q♣|, |3♦|])
* [Taylor ($1000)] (<Hands.HIGH: 1>, [|10♦|, |6♣|])

In [461]:
game.flip()
game

Pot: X | Bet: X | Phase: Flop
Community Cards: [|A♥|, |7♥|, |K♥|]
---------------------------------------
Players: 
* [Robert ($1000)] (<Hands.FLUSH: 6>, [|A♥|, |K♥|, |Q♥|, |8♥|, |7♥|])
* [Lucas ($1000)] (<Hands.PAIR: 2>, [|K♠|, |K♥|, |A♥|, |7♥|, |3♣|])
* [Liz ($1000)] (<Hands.HIGH: 1>, [|A♥|, |K♥|, |Q♦|, |8♦|, |7♥|])
* [Gio ($1000)] (<Hands.HIGH: 1>, [|A♥|, |K♥|, |Q♣|, |7♥|, |3♦|])
* [Taylor ($1000)] (<Hands.HIGH: 1>, [|A♥|, |K♥|, |10♦|, |7♥|, |6♣|])

In [462]:
game.flip()
game


Pot: X | Bet: X | Phase: Turn
Community Cards: [|A♥|, |7♥|, |K♥|, |J♥|]
---------------------------------------
Players: 
* [Robert ($1000)] (<Hands.FLUSH: 6>, [|A♥|, |K♥|, |Q♥|, |J♥|, |8♥|])
* [Lucas ($1000)] (<Hands.PAIR: 2>, [|K♠|, |K♥|, |A♥|, |J♥|, |7♥|])
* [Liz ($1000)] (<Hands.HIGH: 1>, [|A♥|, |K♥|, |Q♦|, |J♥|, |8♦|])
* [Gio ($1000)] (<Hands.HIGH: 1>, [|A♥|, |K♥|, |Q♣|, |J♥|, |7♥|])
* [Taylor ($1000)] (<Hands.HIGH: 1>, [|A♥|, |K♥|, |J♥|, |10♦|, |7♥|])

In [463]:
game.flip()
game

Pot: X | Bet: X | Phase: River
Community Cards: [|A♥|, |7♥|, |K♥|, |J♥|, |6♥|]
---------------------------------------
Players: 
* [Robert ($1000)] (<Hands.FLUSH: 6>, [|A♥|, |K♥|, |Q♥|, |J♥|, |8♥|])
* [Liz ($1000)] (<Hands.FLUSH: 6>, [|A♥|, |K♥|, |J♥|, |7♥|, |6♥|])
* [Lucas ($1000)] (<Hands.FLUSH: 6>, [|A♥|, |K♥|, |J♥|, |7♥|, |6♥|])
* [Gio ($1000)] (<Hands.FLUSH: 6>, [|A♥|, |K♥|, |J♥|, |7♥|, |6♥|])
* [Taylor ($1000)] (<Hands.FLUSH: 6>, [|A♥|, |K♥|, |J♥|, |7♥|, |6♥|])

In [486]:
game.winner()

[Robert ($1000)] (<Hands.FLUSH: 6>, [|A♥|, |K♥|, |Q♥|, |J♥|, |8♥|])